In [24]:
import os, sys, numpy as np
import tensorflow as tf
from keras.utils.np_utils import to_categorical
from tensorflow import keras
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from ipywidgets import interact
from sklearn.model_selection import train_test_split

In [32]:
image = np.load('np_multi/image_arr_154.npy')
mask_ = np.load('np_multi/mask_arr_154.npy')
print(image.shape, mask_.shape)

(132, 32, 192, 192) (132, 32, 192, 192)


In [33]:
# !git clone https://github.com/dhuy228/augmented-volumetric-image-generator

In [27]:
# transpose (0,1,2,3) -> (0,2,3,1) = (86,32,192,192) -> (86,192,192,32)
image2 = np.transpose(image,(0,2,3,1))
mask2_ = np.transpose(mask_,(0,2,3,1))
print(image2.shape, mask2_.shape)

(86, 192, 192, 32) (86, 192, 192, 32)


In [28]:
image = np.expand_dims(image2, axis=-1)
mask_ = np.expand_dims(mask2_, axis=-1)

In [29]:
print(image.shape, mask_.shape)

(86, 192, 192, 32, 1) (86, 192, 192, 32, 1)


In [30]:
from augmented import generator
XX = image
YY = mask_
# XX = np.random.rand(86, 32, 192, 192, 1)
# YY = np.random.rand(86, 32, 192, 192, 1)
print(np.sum(XX))
threshold = 0.5
# XX = XX>threshold
# YY = YY>threshold
print(np.sum(XX))

11713967.742753647
11713967.742753647


In [31]:
%matplotlib inline
def browse_image(images, labels):
#     shape = depth, height, wight
    n = images.shape[2]
    
    def view_image(i):
        
        fig, ax = plt.subplots(nrows = 1, ncols = 2)
        ax[0].imshow(images[...,i], cmap = 'gray', interpolation = 'nearest')
        ax[0].set_title('X Slice: %s' %i)
        ax[1].imshow(labels[...,i], cmap = 'gray', interpolation = 'nearest')
        ax[1].set_title('Y Slice: %s' %i)
    interact(view_image, i = (0,n-1))
browse_image(XX[20,...,0], YY[20,...,0])

interactive(children=(IntSlider(value=15, description='i', max=31), Output()), _dom_classes=('widget-interact'…

In [9]:
S1 = mask_[:30]*1
S2 = mask_[30:50]*2
S3 = mask_[50:]*3
Class_data = np.concatenate((S1,S2,S3),axis=0)
Class_data = to_categorical(Class_data,4,dtype='uint8')

In [20]:
seed = 42
bs = 1
X_train_c, X_valid_c, Y_train_c, Y_valid_c = train_test_split(XX, Class_data, test_size = 0.2, 
                                                              random_state = 20)

In [21]:
print(Y_train_c.shape)
print(np.max(Y_train_c))
print(np.max(X_train_c))
# print(Y_train_c[0])

(68, 192, 192, 32, 4)
1
5.849462509155273


In [12]:
image_aug = generator.customImageDataGenerator(rotation_range = 20)
mask_aug = generator.customImageDataGenerator(rotation_range = 20)

In [16]:
X_train_datagen = image_aug.flow(X_train_c, batch_size=bs, seed=seed)
Y_train_datagen = mask_aug.flow(Y_train_c, batch_size=bs, seed=seed)
train_generator = zip(X_train_datagen, Y_train_datagen)

X_valid_datagen = image_aug.flow(X_valid_c, batch_size=bs, seed=seed,
                                save_to_dir='temp/')
Y_valid_datagen = mask_aug.flow(Y_valid_c, batch_size=bs, seed=seed)
valid_generator = zip(X_valid_datagen, Y_valid_datagen)